In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Python version: 3.6


import os
import copy
import time
import pickle
import numpy as np
from tqdm import tqdm

import torch
from tensorboardX import SummaryWriter

from options import args_parser
from update import LocalUpdate, test_inference
from models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar
from utils import get_dataset, average_weights, exp_details

In [83]:
import argparse

parser = argparse.ArgumentParser()
# parser.add_argument('--name', '-n', default='foo', help='foo')
parser.add_argument('--model', type=str, default='mlp', help='model name(mlp or cnn)')
parser.add_argument('--dataset', type=str, default='mnist', help="name of dataset(mnist or cifar)")
parser.add_argument('--epochs', type=int, default=2, help="number of rounds of training(10)")
parser.add_argument('--iid', type=int, default=1, help='Default set to IID. Set to 0 for non-IID.')
parser.add_argument('--num_users', type=int, default=100, help="number of users: K")
parser.add_argument('--num_classes', type=int, default=10, help="number of classes")
parser.add_argument('--gpu', default=None, help="To use cuda, set to a specific GPU ID. Default set to use CPU.")
parser.add_argument('--frac', type=float, default=0.1, help='the fraction of clients: C')
parser.add_argument('--local_ep', type=int, default=10, help="the number of local epochs: E")
parser.add_argument('--local_bs', type=int, default=10, help="local batch size: B")
parser.add_argument('--lr', type=float, default=0.01, help='learning rate')
parser.add_argument('--optimizer', type=str, default='sgd', help="type of optimizer")
parser.add_argument('--verbose', type=int, default=1, help='verbose')
# parser.add_argument('--seed', type=int, default=1, help='random seed')




# args = parser.parse_args([])
args, _ = parser.parse_known_args()

args
# --dataset=cifar --gpu=0 --iid=0 --epochs=10

Namespace(dataset='mnist', epochs=2, frac=0.1, gpu=None, iid=1, local_bs=10, local_ep=10, lr=0.01, model='mlp', num_classes=10, num_users=100, optimizer='sgd', verbose=1)

In [84]:
# if __name__ == '__main__':
start_time = time.time()

# define paths
path_project = os.path.abspath('..')
logger = SummaryWriter('../logs')

# args = args_parser()
# exp_details(args)

if args.gpu:
    torch.cuda.set_device(args.gpu)
device = 'cuda' if args.gpu else 'cpu'

In [85]:
# load dataset and user groups
train_dataset, test_dataset, user_groups = get_dataset(args)

# BUILD MODEL
if args.model == 'cnn':
    # Convolutional neural netork
    if args.dataset == 'mnist':
        global_model = CNNMnist(args=args)
    elif args.dataset == 'fmnist':
        global_model = CNNFashion_Mnist(args=args)
    elif args.dataset == 'cifar':
        global_model = CNNCifar(args=args)

elif args.model == 'mlp':
    # Multi-layer preceptron
    img_size = train_dataset[0][0].shape
    len_in = 1
    for x in img_size:
        len_in *= x
        global_model = MLP(dim_in=len_in, dim_hidden=64,
                           dim_out=args.num_classes)
else:
    exit('Error: unrecognized model')

In [86]:
# Set the model to train and send it to device.
global_model.to(device)
global_model.train()
print(global_model)

# copy weights
global_weights = global_model.state_dict()

MLP(
  (layer_input): Linear(in_features=784, out_features=64, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=64, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


In [87]:
# Training
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0

for epoch in tqdm(range(args.epochs)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')

    global_model.train()
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)

    for idx in idxs_users:
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger)
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))

    # update global weights
    global_weights = average_weights(local_weights)

    # update global weights
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    global_model.eval()
    for c in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset,
                                  idxs=user_groups[idx], logger=logger)
        acc, loss = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    train_accuracy.append(sum(list_acc)/len(list_acc))

    # print global training loss after every 'i' rounds
    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))

# Test inference after completion of training
test_acc, test_loss = test_inference(args, global_model, test_dataset)

print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

# Saving the objects train_loss and train_accuracy:
file_name = '../save/objects/{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}].pkl'.\
    format(args.dataset, args.model, args.epochs, args.frac, args.iid,
           args.local_ep, args.local_bs)

with open(file_name, 'wb') as f:
    pickle.dump([train_loss, train_accuracy], f)

print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))


  0%|          | 0/2 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/480 (0%)]	Loss: -0.098458
| Global Round : 0 | Local Epoch : 0 | [100/480 (21%)]	Loss: -0.096318
| Global Round : 0 | Local Epoch : 0 | [200/480 (42%)]	Loss: -0.095849
| Global Round : 0 | Local Epoch : 0 | [300/480 (62%)]	Loss: -0.092702
| Global Round : 0 | Local Epoch : 0 | [400/480 (83%)]	Loss: -0.124511
| Global Round : 0 | Local Epoch : 1 | [0/480 (0%)]	Loss: -0.125211
| Global Round : 0 | Local Epoch : 1 | [100/480 (21%)]	Loss: -0.109805
| Global Round : 0 | Local Epoch : 1 | [200/480 (42%)]	Loss: -0.121617
| Global Round : 0 | Local Epoch : 1 | [300/480 (62%)]	Loss: -0.128383
| Global Round : 0 | Local Epoch : 1 | [400/480 (83%)]	Loss: -0.118330
| Global Round : 0 | Local Epoch : 2 | [0/480 (0%)]	Loss: -0.151273
| Global Round : 0 | Local Epoch : 2 | [100/480 (21%)]	Loss: -0.120865
| Global Round : 0 | Local Epoch : 2 | [200/480 (42%)]	Loss: -0.181358
| Global Round : 0 | Local Epoch : 2 | [300/480 (62%)

| Global Round : 0 | Local Epoch : 3 | [400/480 (83%)]	Loss: -0.227608
| Global Round : 0 | Local Epoch : 4 | [0/480 (0%)]	Loss: -0.208594
| Global Round : 0 | Local Epoch : 4 | [100/480 (21%)]	Loss: -0.314753
| Global Round : 0 | Local Epoch : 4 | [200/480 (42%)]	Loss: -0.336143
| Global Round : 0 | Local Epoch : 4 | [300/480 (62%)]	Loss: -0.371156
| Global Round : 0 | Local Epoch : 4 | [400/480 (83%)]	Loss: -0.438820
| Global Round : 0 | Local Epoch : 5 | [0/480 (0%)]	Loss: -0.353381
| Global Round : 0 | Local Epoch : 5 | [100/480 (21%)]	Loss: -0.181792
| Global Round : 0 | Local Epoch : 5 | [200/480 (42%)]	Loss: -0.234974
| Global Round : 0 | Local Epoch : 5 | [300/480 (62%)]	Loss: -0.270388
| Global Round : 0 | Local Epoch : 5 | [400/480 (83%)]	Loss: -0.344310
| Global Round : 0 | Local Epoch : 6 | [0/480 (0%)]	Loss: -0.472446
| Global Round : 0 | Local Epoch : 6 | [100/480 (21%)]	Loss: -0.416918
| Global Round : 0 | Local Epoch : 6 | [200/480 (42%)]	Loss: -0.443179
| Global Round 

| Global Round : 0 | Local Epoch : 7 | [400/480 (83%)]	Loss: -0.557955
| Global Round : 0 | Local Epoch : 8 | [0/480 (0%)]	Loss: -0.493595
| Global Round : 0 | Local Epoch : 8 | [100/480 (21%)]	Loss: -0.581071
| Global Round : 0 | Local Epoch : 8 | [200/480 (42%)]	Loss: -0.444609
| Global Round : 0 | Local Epoch : 8 | [300/480 (62%)]	Loss: -0.606824
| Global Round : 0 | Local Epoch : 8 | [400/480 (83%)]	Loss: -0.288824
| Global Round : 0 | Local Epoch : 9 | [0/480 (0%)]	Loss: -0.527504
| Global Round : 0 | Local Epoch : 9 | [100/480 (21%)]	Loss: -0.516228
| Global Round : 0 | Local Epoch : 9 | [200/480 (42%)]	Loss: -0.440117
| Global Round : 0 | Local Epoch : 9 | [300/480 (62%)]	Loss: -0.507589
| Global Round : 0 | Local Epoch : 9 | [400/480 (83%)]	Loss: -0.680891
| Global Round : 0 | Local Epoch : 0 | [0/480 (0%)]	Loss: -0.100920
| Global Round : 0 | Local Epoch : 0 | [100/480 (21%)]	Loss: -0.085256
| Global Round : 0 | Local Epoch : 0 | [200/480 (42%)]	Loss: -0.095666
| Global Round 

| Global Round : 0 | Local Epoch : 1 | [300/480 (62%)]	Loss: -0.144721
| Global Round : 0 | Local Epoch : 1 | [400/480 (83%)]	Loss: -0.139986
| Global Round : 0 | Local Epoch : 2 | [0/480 (0%)]	Loss: -0.118474
| Global Round : 0 | Local Epoch : 2 | [100/480 (21%)]	Loss: -0.141478
| Global Round : 0 | Local Epoch : 2 | [200/480 (42%)]	Loss: -0.192552
| Global Round : 0 | Local Epoch : 2 | [300/480 (62%)]	Loss: -0.162850
| Global Round : 0 | Local Epoch : 2 | [400/480 (83%)]	Loss: -0.103132
| Global Round : 0 | Local Epoch : 3 | [0/480 (0%)]	Loss: -0.274492
| Global Round : 0 | Local Epoch : 3 | [100/480 (21%)]	Loss: -0.114268
| Global Round : 0 | Local Epoch : 3 | [200/480 (42%)]	Loss: -0.168311
| Global Round : 0 | Local Epoch : 3 | [300/480 (62%)]	Loss: -0.164536
| Global Round : 0 | Local Epoch : 3 | [400/480 (83%)]	Loss: -0.124452
| Global Round : 0 | Local Epoch : 4 | [0/480 (0%)]	Loss: -0.247611
| Global Round : 0 | Local Epoch : 4 | [100/480 (21%)]	Loss: -0.115115
| Global Round 

...

| Global Round : 1 | Local Epoch : 1 | [300/480 (62%)]	Loss: -0.663545
| Global Round : 1 | Local Epoch : 1 | [400/480 (83%)]	Loss: -0.558497
| Global Round : 1 | Local Epoch : 2 | [0/480 (0%)]	Loss: -0.428048
| Global Round : 1 | Local Epoch : 2 | [100/480 (21%)]	Loss: -0.610096
| Global Round : 1 | Local Epoch : 2 | [200/480 (42%)]	Loss: -0.560593
| Global Round : 1 | Local Epoch : 2 | [300/480 (62%)]	Loss: -0.407379
| Global Round : 1 | Local Epoch : 2 | [400/480 (83%)]	Loss: -0.640717
| Global Round : 1 | Local Epoch : 3 | [0/480 (0%)]	Loss: -0.599373
| Global Round : 1 | Local Epoch : 3 | [100/480 (21%)]	Loss: -0.897185
| Global Round : 1 | Local Epoch : 3 | [200/480 (42%)]	Loss: -0.775939
| Global Round : 1 | Local Epoch : 3 | [300/480 (62%)]	Loss: -0.406139
| Global Round : 1 | Local Epoch : 3 | [400/480 (83%)]	Loss: -0.452358
| Global Round : 1 | Local Epoch : 4 | [0/480 (0%)]	Loss: -0.730682
| Global Round : 1 | Local Epoch : 4 | [100/480 (21%)]	Loss: -0.540764
| Global Round 

| Global Round : 1 | Local Epoch : 5 | [200/480 (42%)]	Loss: -0.676459
| Global Round : 1 | Local Epoch : 5 | [300/480 (62%)]	Loss: -0.469678
| Global Round : 1 | Local Epoch : 5 | [400/480 (83%)]	Loss: -0.492454
| Global Round : 1 | Local Epoch : 6 | [0/480 (0%)]	Loss: -0.531860
| Global Round : 1 | Local Epoch : 6 | [100/480 (21%)]	Loss: -0.504845
| Global Round : 1 | Local Epoch : 6 | [200/480 (42%)]	Loss: -0.876556
| Global Round : 1 | Local Epoch : 6 | [300/480 (62%)]	Loss: -0.561541
| Global Round : 1 | Local Epoch : 6 | [400/480 (83%)]	Loss: -0.397569
| Global Round : 1 | Local Epoch : 7 | [0/480 (0%)]	Loss: -0.765637
| Global Round : 1 | Local Epoch : 7 | [100/480 (21%)]	Loss: -0.695936
| Global Round : 1 | Local Epoch : 7 | [200/480 (42%)]	Loss: -0.509804
| Global Round : 1 | Local Epoch : 7 | [300/480 (62%)]	Loss: -0.558616
| Global Round : 1 | Local Epoch : 7 | [400/480 (83%)]	Loss: -0.611555
| Global Round : 1 | Local Epoch : 8 | [0/480 (0%)]	Loss: -0.752571
| Global Round 

100%|██████████| 2/2 [00:42<00:00, 21.44s/it]

 
Avg Training Stats after 2 global rounds:
Training Loss : -0.45299151127226656
Train Accuracy: 80.00% 



 
 Results after 2 global rounds of training:
|---- Avg Train Accuracy: 80.00%
|---- Test Accuracy: 74.16%

 Total Run Time: 45.5068


In [89]:
# PLOTTING (optional)
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

# Plot Loss curve
plt.figure()
plt.title('Training Loss vs Communication rounds')
plt.plot(range(len(train_loss)), train_loss, color='r')
plt.ylabel('Training loss')
plt.xlabel('Communication Rounds')
plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_loss.png'.
            format(args.dataset, args.model, args.epochs, args.frac,
                   args.iid, args.local_ep, args.local_bs))

# Plot Average Accuracy vs Communication rounds
plt.figure()
plt.title('Average Accuracy vs Communication rounds')
plt.plot(range(len(train_accuracy)), train_accuracy, color='k')
plt.ylabel('Average Accuracy')
plt.xlabel('Communication Rounds')
plt.savefig('../save/fed_{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}]_acc.png'.
            format(args.dataset, args.model, args.epochs, args.frac,
                   args.iid, args.local_ep, args.local_bs))


/home/wesleyjtann/miniconda3/envs/fl_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  after removing the cwd from sys.path.
